# WebsocketClientWorker Tutorial

This tutorial is a 2 notebook tutorial. The partner notebook is the notebook entitled `WebsocketServerWorker.ipynb` and is in the same folder as this notebook. You should execute this notebook AFTER you have executed the other one.

In this tutorial, we'll demonstrate how to launch a SocketWorker server which will listen for PyTorch commands over a socket connection. In this tutorial, the two workers are connected via a socket connection on the localhost network.

If you'd prefer to download this notebook and run it locally, you can do so via the `Download .ipynb` in the `File` dropdown field in this Google colab environment. 


# Step -1: Copy This Notebook

Go up to File -> Save A Copy in Drive

This will let you execute the notebook (it won't let you execute this one by default)

# Step 0: Install Dependencies

In [4]:
! git clone https://github.com/OpenMined/PySyft.git
! git checkout dev
! git pull origin dev
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
#!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

import torch

!cd PySyft; pip install -r requirements.txt; python setup.py install

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)

fatal: destination path 'PySyft' already exists and is not an empty directory.
Already on 'dev'
Your branch is up to date with 'origin/dev'.
From https://github.com/OpenMined/PySyft
 * branch              dev        -> FETCH_HEAD
Already up to date.
running install
running bdist_egg
running egg_info
writing syft.egg-info/PKG-INFO
writing dependency_links to syft.egg-info/dependency_links.txt
writing requirements to syft.egg-info/requires.txt
writing top-level names to syft.egg-info/top_level.txt
reading manifest file 'syft.egg-info/SOURCES.txt'
writing manifest file 'syft.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
creating build/bdist.macosx-10.9-x86_64/egg
creating build/bdist.macosx-10.9-x86_64/egg/test
copying build/lib/test/conftest.py -> build/bdist.macosx-10.9-x86_64/egg/test
copying build/lib/test/test_grid.py -> build/bdist.macosx-10.9-x86_64/egg/test
copying build/lib/test/test___init__.py -> build/b

# Step 1: Hook Torch and Create Local Worker

In this step, we hook PyTorch and initialize within the hook a client SocketWorker.

In [22]:
import syft as sy
from syft.workers.websocket_server import WebsocketServerWorker

hook = sy.TorchHook(torch)

local_worker = WebsocketServerWorker(
                            host="localhost",
                            hook=hook,
                            id=0,
                            port=8182,
                            log_msgs=True,
                            verbose=True)

local_worker.start()  # Might need to Interupt with `control-C` or some other means.

W0724 02:22:15.770573 4590904768 hook.py:98] Torch was already hooked... skipping hooking process


RuntimeError: This event loop is already running

In [31]:
hook = sy.TorchHook(torch, local_worker=local_worker)

W0724 02:23:30.960079 4590904768 hook.py:98] Torch was already hooked... skipping hooking process


# Step 2: Create Pointer to Remote Socket Worker

In order to interact with a foreign worker over a socket connection, we need to create a pointer to it containing information on how to contact it. We set the is_pointer=True to signify that this Python object is not in fact a worker in and of itself but that it is merely a pointer to one over the network. We then inform our local worker about this pointer.

In [40]:
from syft.workers.websocket_client import WebsocketClientWorker

remote_client = WebsocketClientWorker(
                            host="localhost",
                            hook=hook,
                            id=2, 
                            port=8181)

hook.local_worker.add_worker(remote_client)

W0724 02:24:37.231598 4590904768 base.py:628] Worker 2 already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:me #objects:0>

# Step 3: Create Tensors & Send To The Worker

In [52]:
x = torch.Tensor([1,2,3,4,5]).send(remote_client)

In [53]:
x2 = torch.Tensor([1,2,3,4,4]).send(remote_client)

In [54]:
x

(Wrapper)>[PointerTensor | me:71068036871 -> 2:59943214158]

In [55]:
x2

(Wrapper)>[PointerTensor | me:1120940095 -> 2:90679127390]

In [66]:
local_worker.get_obj(x)

KeyError: 'Object "(Wrapper)>[PointerTensor | me:71068036871 -> 2:59943214158]" not found on worker!!!You just tried to interact with an object ID:(Wrapper)>[PointerTensor | me:71068036871 -> 2:59943214158] on <WebsocketServerWorker id:0 #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

# Step 4: Execute Operations Like Normal

In [ ]:
x + x

# Step 5: Get Results

In [ ]:
y

In [ ]:
y.get()